In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from builtins import str
from builtins import range
from past.utils import old_div

from coinor.blimpy import PriorityQueue
import time,math,sys

from coinor.grumpy import GenerateRandomMIP
from coinor.grumpy import BranchAndBound
from coinor.grumpy import BBTree
from coinor.grumpy import MOST_FRACTIONAL, FIXED_BRANCHING, PSEUDOCOST_BRANCHING
from coinor.grumpy import DEPTH_FIRST, BEST_FIRST, BEST_ESTIMATE
from coinor.grumpy.BBTree import INFINITY

import numpy as np

from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPModel, CyLPArray

In [2]:
#CONSTRAINTS = ['C0', 'C1']
#VARIABLES = ['x0', 'x1', 'x2'] # Only accept variables in the name of this form: single letter + number
#OBJ = {'x0': 1, 'x1': 1, 'x2': 1}
#MAT = {'x0': [1,1],
#     'x1': [1,1],
#     'x2': [0,1]}
#RHS = [1,2]
CONSTRAINTS, VARIABLES, OBJ, MAT, RHS = GenerateRandomMIP(numVars=40, numCons=30, density=0.1,rand_seed=27)
T = BBTree()

In [3]:
branch_strategy =  PSEUDOCOST_BRANCHING
search_strategy = BEST_ESTIMATE
complete_enumeration = False
display_interval = None
binary_vars = True


if T.get_layout() == 'dot2tex':
    cluster_attrs = {'name':'Key', 'label':r'\text{Key}', 'fontsize':'12'}
    T.add_node('C', label = r'\text{Candidate}', style = 'filled',
                  color = 'yellow', fillcolor = 'yellow')
    T.add_node('I', label = r'\text{Infeasible}', style = 'filled',
                  color = 'orange', fillcolor = 'orange')
    T.add_node('S', label = r'\text{Solution}', style = 'filled',
                  color = 'lightblue', fillcolor = 'lightblue')
    T.add_node('P', label = r'\text{Pruned}', style = 'filled',
                  color = 'red', fillcolor = 'red')
    T.add_node('PC', label = r'\text{Pruned}$\\ $\text{Candidate}', style = 'filled',
                  color = 'red', fillcolor = 'yellow')
else:
    cluster_attrs = {'name':'Key', 'label':'Key', 'fontsize':'12'}
    T.add_node('C', label = 'Candidate', style = 'filled',
                  color = 'yellow', fillcolor = 'yellow')
    T.add_node('I', label = 'Infeasible', style = 'filled',
                  color = 'orange', fillcolor = 'orange')
    T.add_node('S', label = 'Solution', style = 'filled',
                  color = 'lightblue', fillcolor = 'lightblue')
    T.add_node('P', label = 'Pruned', style = 'filled',
                  color = 'red', fillcolor = 'red')
    T.add_node('PC', label = 'Pruned \n Candidate', style = 'filled',
                  color = 'red', fillcolor = 'yellow')
T.add_edge('C', 'I', style = 'invisible', arrowhead = 'none')
T.add_edge('I', 'S', style = 'invisible', arrowhead = 'none')
T.add_edge('S', 'P', style = 'invisible', arrowhead = 'none')
T.add_edge('P', 'PC', style = 'invisible', arrowhead = 'none')
T.create_cluster(['C', 'I', 'S', 'P', 'PC'], cluster_attrs)

# Change to CyLP format
cyOBJ = CyLPArray([val for val in OBJ.values()]) # CyLP takes min
cyMAT = np.matrix([MAT[v] for v in VARIABLES]).T
cyRHS = CyLPArray(RHS)


# The initial lower bound
LB = -INFINITY
# The number of LP's solved, and the number of nodes solved
node_count = 1
iter_count = 0
lp_count = 0

numCons = len(CONSTRAINTS)
numVars = len(VARIABLES)
# List of incumbent solution variable values
opt = dict([(i, 0) for i in VARIABLES])
pseudo_u = dict((i, (OBJ[i], 0)) for i in VARIABLES)
pseudo_d = dict((i, (OBJ[i], 0)) for i in VARIABLES)
print("===========================================")
print("Starting Branch and Bound")
if branch_strategy == MOST_FRACTIONAL:
    print("Most fractional variable")
elif branch_strategy == FIXED_BRANCHING:
    print("Fixed order")
elif branch_strategy == PSEUDOCOST_BRANCHING:
    print("Pseudocost brancing")
else:
    print("Unknown branching strategy %s" %branch_strategy)
if search_strategy == DEPTH_FIRST:
    print("Depth first search strategy")
elif search_strategy == BEST_FIRST:
    print("Best first search strategy")
else:
    print("Unknown search strategy %s" %search_strategy)
print("===========================================")
# List of candidate nodes
Q = PriorityQueue()
# The current tree depth
cur_depth = 0
cur_index = 0
# Timer
timer = time.time()
Q.push(0, -INFINITY, (0, None, None, None, None, None, None))
# Branch and Bound Loop
while not Q.isEmpty():
    infeasible = False
    integer_solution = False
    (cur_index, parent, relax, branch_var, branch_var_value, sense,
    rhs) = Q.pop()
    if cur_index is not 0:
        cur_depth = T.get_node_attr(parent, 'level') + 1
    else:
        cur_depth = 0
    print("")
    print("----------------------------------------------------")
    print("")
    if LB > -INFINITY:
        print("Node: %s, Depth: %s, LB: %s" %(cur_index,cur_depth,LB))
    else:
        print("Node: %s, Depth: %s, LB: %s" %(cur_index,cur_depth,"None"))
    if relax is not None and relax <= LB:
        print("Node pruned immediately by bound")
        T.set_node_attr(parent, 'color', 'red')
        continue
    #====================================
    #    LP Relaxation
    #====================================
    # Compute lower bound by LP relaxation
    prob = CyLPModel()
    
    # CyLP do not allow change variable object without model
    if binary_vars:
        var = prob.addVariable('x', dim=len(VARIABLES))
        prob += 0 <= var <= 1
    else:
        var = prob.addVariable('x', dim=len(VARIABLES))
        prob += 0 <= var # To avoid random generated problems be unbounded
    
    
    prob += cyMAT * var <= RHS
    prob.objective = -cyOBJ * var
    s = CyClpSimplex(prob)
    
    # Fix all prescribed variables
    branch_vars = []
    if cur_index is not 0:
        sys.stdout.write("Branching variables: ")
        branch_vars.append(branch_var)
        if sense == '>=':
            prob += var[int(branch_var[1:])] >= rhs # slice the name for index
        else:
            prob += var[int(branch_var[1:])] <= rhs # slice the name for index
        print(branch_var, end=' ')
        pred = parent
        while not str(pred) == '0':
            pred_branch_var = T.get_node_attr(pred, 'branch_var')
            pred_rhs = T.get_node_attr(pred, 'rhs')
            pred_sense = T.get_node_attr(pred, 'sense')
            if pred_sense == '<=':
                prob += var[int(pred_branch_var[1:])] <= pred_rhs
            else:
                prob += var[int(pred_branch_var[1:])] >= pred_rhs
            print(pred_branch_var, end=' ')
            branch_vars.append(pred_branch_var)
            pred = T.get_node_attr(pred, 'parent')
        print()
    # Solve the LP relaxation
    s = CyClpSimplex(prob)
    s.initialSolve()
    print(s.getStatusCode())
    lp_count = lp_count +1
    
    if (s.getStatusCode() < 0):
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        print(lp_count)
        print(s.getStatusCode())
        print(s.primal())
        print(s.objectiveValue)
        print(s.primalVariableSolution)
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    
    # Check infeasibility
    #-1 - unknown e.g. before solve or if postSolve says not optimal
    #0 - optimal
    #1 - primal infeasible
    #2 - dual infeasible
    #3 - stopped on iterations or time
    #4 - stopped due to errors
    #5 - stopped by event handler (virtual int ClpEventHandler::event())
    infeasible = (s.getStatusCode() in [-1,1,3,4,5])
    # Print status
    if infeasible:
        print("LP Solved, status: Infeasible")
    else:
        print("LP Solved, status: %s, obj: %s" %(s.getStatusCode(),-s.objectiveValue))
        
    if(s.getStatusCode() == 0):
        relax = -s.objectiveValue
        # Update pseudocost
        if branch_var != None:
            if sense == '<=':
                pseudo_d[branch_var] = (
                old_div((pseudo_d[branch_var][0]*pseudo_d[branch_var][1] +
                old_div((T.get_node_attr(parent, 'obj') - relax),
                (branch_var_value - rhs))),(pseudo_d[branch_var][1]+1)),
                pseudo_d[branch_var][1]+1)
            else:
                pseudo_u[branch_var] = (
                old_div((pseudo_u[branch_var][0]*pseudo_d[branch_var][1] +
                 old_div((T.get_node_attr(parent, 'obj') - relax),
                 (rhs - branch_var_value))),(pseudo_u[branch_var][1]+1)),
                pseudo_u[branch_var][1]+1)
        var_values = dict([(i, s.primalVariableSolution['x'][int(i[1:])]) for i in VARIABLES])
        integer_solution = 1
        for i in VARIABLES:
            if (abs(round(var_values[i]) - var_values[i]) > .001):
                integer_solution = 0
                break
        # Determine integer_infeasibility_count and
        # Integer_infeasibility_sum for scatterplot and such
        integer_infeasibility_count = 0
        integer_infeasibility_sum = 0.0
        for i in VARIABLES:
            if (var_values[i] not in set([0,1])):
                integer_infeasibility_count += 1
                integer_infeasibility_sum += min([var_values[i],
                                                  1.0-var_values[i]])
        if (integer_solution and relax>LB):
            LB = relax
            for i in VARIABLES:
                # These two have different data structures first one
                #list, second one dictionary
                opt[i] = var_values[i]
            print("New best solution found, objective: %s" %relax)
            for i in VARIABLES:
                if var_values[i] > 0:
                    print("%s = %s" %(i, var_values[i]))
        elif (integer_solution and relax<=LB):
            print("New integer solution found, objective: %s" %relax)
            for i in VARIABLES:
                if var_values[i] > 0:
                    print("%s = %s" %(i, var_values[i]))
        else:
            print("Fractional solution:")
            for i in VARIABLES:
                if var_values[i] > 0:
                    print("%s = %s" %(i, var_values[i]))
        #For complete enumeration
        if complete_enumeration:
            relax = LB - 1
    else:
        relax = INFINITY
    if integer_solution:
        print("Integer solution")
        BBstatus = 'S'
        status = 'integer'
        color = 'lightblue'
    elif infeasible:
        print("Infeasible node")
        BBstatus = 'I'
        status = 'infeasible'
        color = 'orange'
    elif not complete_enumeration and relax <= LB:
        print("Node pruned by bound (obj: %s, UB: %s)" %(relax,LB))
        BBstatus = 'P'
        status = 'fathomed'
        color = 'red'
    elif cur_depth >= numVars :
        print("Reached a leaf")
        BBstatus = 'fathomed'
        status = 'L'
    else:
        BBstatus = 'C'
        status = 'candidate'
        color = 'yellow'
    if BBstatus is 'I':
        if T.get_layout() == 'dot2tex':
            label = '\text{I}'
        else:
            label = 'I'
    else:
        label = "%.1f"%relax
    if iter_count == 0:
        if status is not 'candidate':
            integer_infeasibility_count = None
            integer_infeasibility_sum = None
        if status is 'fathomed':
            if T._incumbent_value is None:
                print('WARNING: Encountered "fathom" line before '+\
                    'first incumbent.')
        T.AddOrUpdateNode(0, None, None, 'candidate', relax,
                         integer_infeasibility_count,
                         integer_infeasibility_sum,
                         label = label,
                         obj = relax, color = color,
                         style = 'filled', fillcolor = color)
        if status is 'integer':
            T._previous_incumbent_value = T._incumbent_value
            T._incumbent_value = relax
            T._incumbent_parent = -1
            T._new_integer_solution = True
#           #Currently broken
#           if ETREE_INSTALLED and T.attr['display'] == 'svg':
#               T.write_as_svg(filename = "node%d" % iter_count,
#                                 nextfile = "node%d" % (iter_count + 1),
#                                 highlight = cur_index)
    else:
        _direction = {'<=':'L', '>=':'R'}
        if status is 'infeasible':
            integer_infeasibility_count = T.get_node_attr(parent,
                                 'integer_infeasibility_count')
            integer_infeasibility_sum = T.get_node_attr(parent,
                                 'integer_infeasibility_sum')
            relax = T.get_node_attr(parent, 'lp_bound')
        elif status is 'fathomed':
            if T._incumbent_value is None:
                print('WARNING: Encountered "fathom" line before'+\
                    ' first incumbent.')
                print('  This may indicate an error in the input file.')
        elif status is 'integer':
            integer_infeasibility_count = None
            integer_infeasibility_sum = None
        T.AddOrUpdateNode(cur_index, parent, _direction[sense],\
                             status, relax,\
                             integer_infeasibility_count,\
                             integer_infeasibility_sum,\
                             branch_var = branch_var,\
                             branch_var_value = var_values[branch_var],\
                             sense = sense, rhs = rhs, obj = relax,\
                             color = color, style = 'filled',\
                             label = label, fillcolor = color)
        if status is 'integer':
            T._previous_incumbent_value = T._incumbent_value
            T._incumbent_value = relax
            T._incumbent_parent = parent
            T._new_integer_solution = True
        # Currently Broken
#           if ETREE_INSTALLED and T.attr['display'] == 'svg':
#               T.write_as_svg(filename = "node%d" % iter_count,
#                                 prevfile = "node%d" % (iter_count - 1),
#                                 nextfile = "node%d" % (iter_count + 1),
#                                 highlight = cur_index)
        if T.get_layout() == 'dot2tex':
            _dot2tex_label = {'>=':' \geq ', '<=':' \leq '}
            T.set_edge_attr(parent, cur_index, 'label',\
                               str(branch_var) + _dot2tex_label[sense] +\
                               str(rhs))
        else:
            T.set_edge_attr(parent, cur_index, 'label',\
                               str(branch_var) + sense + str(rhs))
    iter_count += 1
    if BBstatus == 'C':
        # Branching:
        # Choose a variable for branching
        branching_var = None
        if branch_strategy == FIXED_BRANCHING:
            #fixed order
            for i in VARIABLES:
                frac = min(s.primalVariableSolution['x'][int(i[1:])]-math.floor(s.primalVariableSolution['x'][int(i[1:])]),\
                           math.ceil(s.primalVariableSolution['x'][int(i[1:])]) - s.primalVariableSolution['x'][int(i[1:])])
                if (frac > 0):
                    min_frac = frac
                    branching_var = i
                    # TODO(aykut): understand this break
                    break
        elif branch_strategy == MOST_FRACTIONAL:
            #most fractional variable
            min_frac = -1
            for i in VARIABLES:
                frac = min(s.primalVariableSolution['x'][int(i[1:])]-math.floor(s.primalVariableSolution['x'][int(i[1:])]),\
                           math.ceil(s.primalVariableSolution['x'][int(i[1:])])- s.primalVariableSolution['x'][int(i[1:])])
                if (frac> min_frac):
                    min_frac = frac
                    branching_var = i
        elif branch_strategy == PSEUDOCOST_BRANCHING:
            scores = {}
            for i in VARIABLES:
                # find the fractional solutions
                if (s.primalVariableSolution['x'][int(i[1:])] - math.floor(s.primalVariableSolution['x'][int(i[1:])])) != 0:
                    scores[i] = min(pseudo_u[i][0]*(1-s.primalVariableSolution['x'][int(i[1:])]),\
                                    pseudo_d[i][0]*s.primalVariableSolution['x'][int(i[1:])])
                # sort the dictionary by value
            branching_var = sorted(list(scores.items()),
                                   key=lambda x : x[1])[-1][0]
        else:
            print("Unknown branching strategy %s" %branch_strategy)
            exit()
        if branching_var is not None:
            print("Branching on variable %s" %branching_var)
        #Create new nodes
        if search_strategy == DEPTH_FIRST:
            priority = (-cur_depth - 1, -cur_depth - 1)
        elif search_strategy == BEST_FIRST:
            priority = (-relax, -relax)
        elif search_strategy == BEST_ESTIMATE:
            priority = (-relax - pseudo_d[branching_var][0]*\
                             (math.floor(s.primalVariableSolution['x'][int(branching_var[1:])]) -\
                                  s.primalVariableSolution['x'][int(branching_var[1:])]),\
                        -relax + pseudo_u[branching_var][0]*\
                             (math.ceil(s.primalVariableSolution['x'][int(branching_var[1:])]) -\
                                  s.primalVariableSolution['x'][int(branching_var[1:])]))
        node_count += 1
        Q.push(node_count, priority[0], (node_count, cur_index, relax, branching_var,
                var_values[branching_var],
                '<=', math.floor(s.primalVariableSolution['x'][int(branching_var[1:])])))
        node_count += 1
        Q.push(node_count, priority[1], (node_count, cur_index, relax, branching_var,
                var_values[branching_var],
                '>=', math.ceil(s.primalVariableSolution['x'][int(branching_var[1:])])))
        T.set_node_attr(cur_index, color, 'green')
    if T.root is not None and display_interval is not None and\
            iter_count%display_interval == 0:
        T.display(count=iter_count)

timer = int(math.ceil((time.time()-timer)*1000))
print("")
print("===========================================")
print("Branch and bound completed in %sms" %timer)
print("Strategy: %s" %branch_strategy)
if complete_enumeration:
    print("Complete enumeration")
print("%s nodes visited " %node_count)
print("%s LP's solved" %lp_count)
print("===========================================")
print("Optimal solution")
#print optimal solution
for i in sorted(VARIABLES):
    if opt[i] > 0:
        print("%s = %s" %(i, opt[i]))
print("Objective function value")
print(LB)
print("===========================================")
if T.attr['display'] is not 'off':
    T.display(count=iter_count)
T._lp_count = lp_count

Starting Branch and Bound
Pseudocost brancing
Unknown search strategy Best Estimate

----------------------------------------------------

Node: 0, Depth: 0, LB: None
0
LP Solved, status: 0, obj: 190.162037037037
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 0.875
x4 = 1.0
x5 = 1.0
x6 = 0.2962962962962966
x7 = 1.0
x8 = 0.41666666666666646
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 0.6250000000000002
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 0.11111111111111122
x34 = 1.0
x35 = 0.27777777777777757
x36 = 1.0
x37 = 1.0
x39 = 1.0
adding root 0
Branching on variable x8

----------------------------------------------------

Node: 2, Depth: 1, LB: None
Branching variables: x8 
0
LP Solved, status: 0, obj: 189.88888888888889
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 0.5000000000000002
x4 = 1.0
x5 = 1.0
x6 = 0.6666666666666667
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0

0
LP Solved, status: 0, obj: 186.60317460317458
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x13 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 0.7142857142857146
x23 = 1.0
x24 = 0.5555555555555556
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 0.11111111111111122
x34 = 1.0
x35 = 0.27777777777777757
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x33

----------------------------------------------------

Node: 40, Depth: 5, LB: None
Branching variables: x33 x13 x6 x3 x8 
0
LP Solved, status: 0, obj: 186.38095238095238
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 0.11111111111111122
x13 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 0.7142857142857146
x23 = 1.0
x24 = 0.5555555555555556
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0

Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.8750000000000001
x21 = 1.0
x22 = 0.2571428571428574
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 0.25
x34 = 1.0
x36 = 1.0
x37 = 1.0
x38 = 0.8
x39 = 1.0
Branching on variable x33

----------------------------------------------------

Node: 32, Depth: 9, LB: None
Branching variables: x13 x33 x35 x32 x9 x24 x6 x3 x8 
0
LP Solved, status: 0, obj: 185.13333333333333
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 0.11111111111111122
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 0.6000000000000004
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x38 = 0.2
x39 = 1.0
Branching on variable x22

------------

Branching on variable x24

----------------------------------------------------

Node: 80, Depth: 6, LB: None
Branching variables: x12 x33 x21 x27 x3 x8 
0
LP Solved, status: 0, obj: 184.30357142857144
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0000000000000002
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x22 = 0.1785714285714286
x23 = 1.0
x24 = 0.6666666666666664
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x24

----------------------------------------------------

Node: 10, Depth: 4, LB: None
Branching variables: x24 x6 x3 x8 
0
LP Solved, status: 0, obj: 186.66984126984124
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x13 = 0.6000000000000001
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.

Branching variables: x37 x20 x8 
0
LP Solved, status: 0, obj: 185.85
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.25
x3 = 0.5000000000000002
x4 = 1.0
x5 = 1.0
x7 = 1.0
x8 = 1.0
x9 = 0.7999999999999999
x10 = 1.0
x11 = 1.0
x12 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 0.9333333333333332
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 1.0
x34 = 1.0
x36 = 1.0
x39 = 1.0
Branching on variable x3

----------------------------------------------------

Node: 140, Depth: 4, LB: None
Branching variables: x3 x37 x20 x8 
0
LP Solved, status: 0, obj: 184.5642857142857
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.25
x4 = 1.0
x5 = 1.0
x7 = 1.0
x8 = 1.0
x9 = 0.7999999999999999
x10 = 1.0
x11 = 1.0
x12 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x21 = 1.0
x22 = 0.7142857142857146
x23 = 1.0
x24 = 0.9333333333333332
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 1.0
x34 = 1.0
x

0
LP Solved, status: 0, obj: 183.51388888888889
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 0.8333333333333335
x5 = 1.0
x6 = 0.11111111111111147
x7 = 1.0
x8 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x12 = 0.8888888888888888
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 0.6666666666666664
x26 = 1.0
x27 = 0.09999999999999998
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x34 = 1.0
x36 = 1.0
x39 = 1.0
Node pruned by bound (obj: 183.51388888888889, UB: 184.0)

----------------------------------------------------

Node: 125, Depth: 7, LB: 184.0
Branching variables: x22 x9 x13 x24 x6 x3 x8 
0
LP Solved, status: 0, obj: 183.55555555555554
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x13 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 1.0
x23 = 1.0
x26 = 1.0
x27 = 0.8
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 0.111

1
LP Solved, status: Infeasible
Infeasible node

----------------------------------------------------

Node: 60, Depth: 10, LB: 184.0
Branching variables: x27 x22 x13 x35 x32 x9 x24 x6 x3 x8 
0
LP Solved, status: 0, obj: 181.52777777777777
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.6388888888888888
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.3333333333333333
x26 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 0.11111111111111122
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 181.52777777777777, UB: 184.0)

----------------------------------------------------

Node: 107, Depth: 10, LB: 184.0
Branching variables: x21 x27 x22 x35 x12 x33 x13 x6 x3 x8 
0
LP Solved, status: 0, obj: 179.08333333333331
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.75
x4 = 1.0
x5 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x13 = 1.0
x14 = 1.0
x15 

0
LP Solved, status: 0, obj: 181.70833333333331
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.8750000000000001
x22 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.3333333333333333
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 0.25
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 181.70833333333331, UB: 184.0)

----------------------------------------------------

Node: 148, Depth: 11, LB: 184.0
Branching variables: x27 x22 x13 x33 x35 x32 x9 x24 x6 x3 x8 
0
LP Solved, status: 0, obj: 180.66666666666669
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 0.11111111111111122
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.3333333333333333
x26 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0

Branching on variable x2

----------------------------------------------------

Node: 173, Depth: 6, LB: 184.0
Branching variables: x2 x33 x21 x27 x3 x8 
0
LP Solved, status: 0, obj: 183.20357142857142
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0000000000000002
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.19999999999999996
x22 = 0.1785714285714286
x23 = 1.0
x24 = 0.6666666666666664
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 0.75
x32 = 1.0
x33 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 183.20357142857142, UB: 184.0)

----------------------------------------------------

Node: 172, Depth: 6, LB: 184.0
Branching variables: x2 x33 x21 x27 x3 x8 
0
LP Solved, status: 0, obj: 182.70357142857142
Fractional solution:
x0 = 1.0
x1 = 1.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0000000000000002
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x14 = 1.0
x15 = 1.0
x16 = 1

In [4]:
T2 = BBTree()
BranchAndBound(T2, CONSTRAINTS, VARIABLES, OBJ, MAT, RHS,
               branch_strategy = branch_strategy,
               search_strategy = search_strategy,
               complete_enumeration = False,
               display_interval = None,
               binary_vars = True)

Starting Branch and Bound
Pseudocost brancing
Unknown search strategy Best Estimate

----------------------------------------------------

Node: 0, Depth: 0, LB: None
LP Solved, status: Optimal, obj: 190.16203706999997
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 0.875
x4 = 1.0
x5 = 1.0
x6 = 0.2962963
x7 = 1.0
x8 = 0.41666667
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 0.625
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 0.11111111
x34 = 1.0
x35 = 0.27777778
x36 = 1.0
x37 = 1.0
x39 = 1.0
adding root 0
Branching on variable x8

----------------------------------------------------

Node: 2, Depth: 1, LB: None
Branching variables: x8 
LP Solved, status: Optimal, obj: 189.88888889999998
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 0.5
x4 = 1.0
x5 = 1.0
x6 = 0.66666667
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x

x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.96875
x21 = 0.20833333
x23 = 1.0
x24 = 0.66666667
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 0.14583333
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x21

----------------------------------------------------

Node: 28, Depth: 8, LB: None
Branching variables: x33 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 185.39047618
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 0.11111111
x13 = 0.1
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 0.97142857
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 0.71428571
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x13

----------------------------------------------------

Node: 8, Depth: 3,

Branching variables: x22 x13 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 185.52777777
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.63888889
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.33333333
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 0.11111111
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x25

----------------------------------------------------

Node: 62, Depth: 10, LB: None
Branching variables: x25 x22 x13 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 185.39444443999997
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.63888889
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 0.11111111
x34 = 1.0
x35 = 1.

LP Solved, status: Optimal, obj: 184.75
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.75
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x2

----------------------------------------------------

Node: 97, Depth: 14, LB: None
Branching variables: x2 x38 x12 x33 x25 x22 x13 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 183.5
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 0.75
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Branching on variable x31

----------------------------------------------------

Node: 57,

LP Solved, status: Optimal, obj: 184.12103176
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 0.83333333
x5 = 1.0
x6 = 0.11111111
x7 = 1.0
x8 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x12 = 0.88888889
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 0.60714286
x23 = 1.0
x24 = 0.66666667
x26 = 1.0
x27 = 0.1
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x34 = 1.0
x36 = 1.0
x39 = 1.0
Branching on variable x22

----------------------------------------------------

Node: 39, Depth: 6, LB: None
Branching variables: x13 x9 x24 x6 x3 x8 
LP Solved, status: Infeasible
Infeasible node

----------------------------------------------------

Node: 110, Depth: 12, LB: None
Branching variables: x25 x12 x22 x13 x33 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 184.2
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 

...

x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 0.75
x32 = 1.0
x33 = 0.5
x34 = 1.0
x36 = 1.0
x39 = 1.0
Node pruned by bound (obj: 181.31428569, UB: 184.0)

----------------------------------------------------

Node: 144, Depth: 8, LB: 184.0
Node pruned immediately by bound

----------------------------------------------------

Node: 67, Depth: 12, LB: 184.0
Branching variables: x12 x33 x25 x22 x13 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 182.35
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 0.75
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.2
x21 = 1.0
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x38 = 0.2
x39 = 1.0
Node pruned by bound (obj: 182.35, UB: 184.0)

----------------------------------------------------

Node: 15, Depth: 3, LB: 184.0
Branching variables: x33 x20 x8 
LP Solved, status:

LP Solved, status: Optimal, obj: 181.73333333
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x12 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.2
x21 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.33333333
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 181.73333333, UB: 184.0)

----------------------------------------------------

Node: 81, Depth: 6, LB: 184.0
Branching variables: x12 x33 x21 x27 x3 x8 
LP Solved, status: Optimal, obj: 181.70357145000003
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 0.875
x12 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 0.2
x22 = 0.17857143
x23 = 1.0
x24 = 0.66666667
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x34 = 1.0
x35 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned b

x10 = 1.0
x11 = 1.0
x12 = 0.11111111
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 1.0
x23 = 1.0
x24 = 1.0
x25 = 0.33333333
x26 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 180.66666666, UB: 184.0)

----------------------------------------------------

Node: 160, Depth: 7, LB: 184.0
Branching variables: x3 x2 x27 x0 x33 x20 x8 
LP Solved, status: Optimal, obj: 180.59999998
Fractional solution:
x0 = 1.0
x1 = 1.0
x4 = 1.0
x5 = 1.0
x7 = 1.0
x8 = 1.0
x9 = 0.8
x10 = 1.0
x11 = 1.0
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 1.0
x19 = 1.0
x20 = 1.0
x21 = 1.0
x22 = 1.0
x23 = 1.0
x24 = 0.93333333
x26 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x32 = 1.0
x33 = 1.0
x35 = 0.5
x36 = 1.0
x39 = 1.0
Node pruned by bound (obj: 180.59999998, UB: 184.0)

----------------------------------------------------

Node: 106, Depth: 10, LB: 184.0
Branching variables: x21 x27 x22 x35 x12 x33 x13 x6 x3 x

Branching variables: x2 x33 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 183.45714285000003
Fractional solution:
x0 = 1.0
x1 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x13 = 0.1
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 0.97142857
x19 = 1.0
x20 = 0.2
x21 = 1.0
x22 = 0.71428571
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 = 1.0
x33 = 1.0
x34 = 1.0
x36 = 1.0
x37 = 1.0
x39 = 1.0
Node pruned by bound (obj: 183.45714285000003, UB: 184.0)

----------------------------------------------------

Node: 171, Depth: 9, LB: 184.0
Branching variables: x2 x33 x35 x32 x9 x24 x6 x3 x8 
LP Solved, status: Optimal, obj: 180.94464285
Fractional solution:
x0 = 1.0
x1 = 1.0
x2 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x9 = 1.0
x10 = 1.0
x11 = 1.0
x13 = 0.1
x14 = 1.0
x15 = 1.0
x16 = 1.0
x17 = 1.0
x18 = 0.97142857
x19 = 1.0
x20 = 0.2875
x21 = 1.0
x22 = 0.71428571
x23 = 1.0
x24 = 1.0
x26 = 1.0
x27 = 1.0
x28 = 1.0
x29 = 1.0
x30 = 1.0
x31 

({'x0': 1.0,
  'x1': 1.0,
  'x2': 1.0,
  'x3': 0.0,
  'x4': 1.0,
  'x5': 1.0,
  'x6': 0.0,
  'x7': 1.0,
  'x8': 0.0,
  'x9': 1.0,
  'x10': 1.0,
  'x11': 1.0,
  'x12': 0.0,
  'x13': 0.0,
  'x14': 1.0,
  'x15': 1.0,
  'x16': 1.0,
  'x17': 1.0,
  'x18': 1.0,
  'x19': 1.0,
  'x20': 1.0,
  'x21': 1.0,
  'x22': 0.0,
  'x23': 1.0,
  'x24': 1.0,
  'x25': 0.0,
  'x26': 1.0,
  'x27': 1.0,
  'x28': 1.0,
  'x29': 1.0,
  'x30': 1.0,
  'x31': 1.0,
  'x32': 1.0,
  'x33': 0.0,
  'x34': 1.0,
  'x35': 0.0,
  'x36': 1.0,
  'x37': 1.0,
  'x38': 1.0,
  'x39': 1.0},
 184.0)